Para el ajax en enlima.pe https://www.enlima.pe/calendario-cultural/dia/2020-01-14?page=1

In [14]:
import requests
from bs4 import BeautifulSoup

import datetime
from datetime import timedelta

import re
import time

import pandas as pd

La siguiente será una función que me permitirá hacer los links de acuerdo a la estructura de la página. Es decir, tendré una lista de links correspondientes a cada día.

In [2]:
def pass_datetimes_make_links(initial_time, final_time):
    # A este función se le deben de pasar dos objetos datetime.datetime especificados respectivamente
    date_list = [] #Aca pondre las fechas que generaré
    time_obj = initial_time #Es mi tiempo inicial
    while time_obj <= final_time:  #Si tengo la fecha inicial menor que la final
        fecha_formateada = time_obj.strftime("%Y-%m-%d") #formateo mi tiempo
        date_list.append(fecha_formateada)
        time_obj += timedelta(days=1) #Voy aumentando los días hasta que cumpla la condicion
    #Haré los links finalmente y los pondré denuevo en una lista.
    lista_links_dias = ['https://www.enlima.pe/calendario-cultural/dia/'+fecha+'?page=1' for fecha in date_list]
    return lista_links_dias

#Esto me botará una lista con todos los links padres a los que entraré primero.

In [3]:
#Tengo una lista de links en general de los dias
def scrap_me_internal_links(lista_links_dias):
    #creo el diccionario final
    dict_fecha_links_internos = {}
    #Para cada dia dentro de la lista de los links del dia
    for link_del_dia in lista_links_dias:
        time.sleep(1)
        #crea la conexion
        response = requests.get(link_del_dia)
        soup = BeautifulSoup(response.text,"html.parser")
        #crea una lista para los links de los eventos del dia
        lista_links_eventos_del_dia = []
        #para cada coneccions, hay varios links internos
        for i in soup.find('div',{'class':'innera'}).find_all('td',{'class':'views-field views-field-title views-align-left'}):
            #Pondré esos links internos en la lista anteriormente creda
            for a in i.find_all('a',href=True):
                lista_links_eventos_del_dia.append('https://www.enlima.pe' + str(a['href']))
            #hasta acá tengo una lista de todos los links internos por dia.
        dict_por_dia = scrap_me_los_eventos(lista_links_eventos_del_dia,i=0)
        #Para toda la data generada por dia, ponmela en un diccionario aparte.
        dict_fecha_links_internos[link_del_dia[46:56]] = dict_por_dia
    return dict_fecha_links_internos

In [4]:
#Esta funcion está hecha para scrapear una lista de links internos (de eventos) y
#botar la data que se obtenga dentro de un dict. 
def scrap_me_los_eventos(lista_links_eventos_del_dia,i=0):
    dict_scrap_final = {}
    dict_scrap_interno = {}
    for link_evento in lista_links_eventos_del_dia:
        i+=1
        response = requests.get(link_evento)
        soup = BeautifulSoup(response.text,"html.parser")
        titulo = soup.find('h1',{'class':'page__title title'}).text
        basic_info = [div.text for div in soup.find_all('div',{'class':'field-item even'})[2:7]]
        tipo, lugar, intervalo_fecha, precio, descripcion_basica = map(str,basic_info)
        to_parse = [elem.text for elem in soup.find('div',{'class':
                                                           'ds-1col node node-lugar view-mode-reference clearfix'}).find_all('div',
                                                                                                                             {'class':'field-item even'})]
        pattern = re.compile(r'-\d*.\d*,-\d*.\d*',re.MULTILINE|re.DOTALL) #Tengo el patron de las coordenadas
        script = soup.find_all('script',text=pattern) #saco el script que contiene el patron
        try: 
            coordenadas_list = re.findall(pattern,str(script))[0].split(',') #Lo obtengo y lo pongo dentro de una lista.
        except:
            coordenadas_list = ['nohay','nohay']
        dict_scrap_final['evento'+str(i)] = {'nombre_evento':titulo,'tipo_evento':tipo,'lugar_evento':lugar,
                                        'intervalo_fecha':intervalo_fecha,'precio':precio,
                                         'descripcion_basica':descripcion_basica,'data_extra':to_parse,
                                         'coordenadas':coordenadas_list}
    return dict_scrap_final

In [5]:
#Genero segun dos dias:
initial_time = datetime.datetime(2019,1,2)
final_time = datetime.datetime(2019,1,4)

In [6]:
lista_links_dias = pass_datetimes_make_links(initial_time,final_time)
dict_final3 = scrap_me_internal_links(lista_links_dias)

In [45]:
for k,v in dict_final3.items():
    df = pd.DataFrame().from_dict(v,orient='index')
    df.to_csv('enlima_'+k+'.csv',index=False)

In [47]:
df1 = pd.read_csv('enlima_2019-01-02.csv', encoding='ISO-8859-1')
df1

,nombre_evento,tipo_evento,lugar_evento,intervalo_fecha,precio,descripcion_basica,data_extra,coordenadas
0,El asesinato de la familia Borden,Cine,Varias sedes - Lima,Del 20.12.2018 al 03.01.2019,S/. 20.00,ChloÃ« Sevigny y Kristen Stewart protagonizan ...,"['Varias sedes - Lima', 'Ãrea metropolitana',...","['nohay', 'nohay']"
1,Papeles sobre el suelo,ExposiciÃ³n,Instituto Italiano de Cultura,9:00 am,Del 07.12.2018 al 09.02.2019,GRATIS,"['Instituto Italiano de Cultura', 'Av. Arequip...",['-77.0357290999999975156242726370692253112792...
2,Caleidoscopio de un Arenal,ExposiciÃ³n,Centro Cultural de Bellas Artes,10:00 am,Del 13.12.2018 al 27.01.2019,GRATIS,"['Centro Cultural de Bellas Artes', 'Jr. Huall...",['-77.0301716000000027406713343225419521331787...
3,Liliane Tomasko,ExposiciÃ³n,MATE Lab - Museo Mario Testino,10:00 am,Del 25.11.2018 al 27.01.2019,GRATIS,"['MATE Lab - Museo Mario Testino', 'Av. Pedro ...",['-77.0228443000000027041096473112702369689941...
4,"Dora Mayer, la perseverancia femenina frente a...",ExposiciÃ³n,Centro Cultural Inca Garcilaso,10:00 am,Del 06.12.2018 al 27.01.2019,GRATIS,"['Centro Cultural Inca Garcilaso', 'Jr. Ucayal...",['-77.0290530999999987216142471879720687866210...
5,Daido Tokyo Color,ExposiciÃ³n,MATE - Museo Mario Testino,10:00 am,Del 19.10.2018 al 03.03.2019,GRATIS,"['MATE - Museo Mario Testino', 'Av. Pedro de O...",['-77.0228443000000027041096473112702369689941...
6,Isla Azul. LÃ¡minas de Iquitos,Centro Cultural Inca Garcilaso,10:00 am,Del 13.12.2018 al 27.01.2019,GRATIS,"Iquitos fue la ciudad-puerto protagonista, jun...","['Centro Cultural Inca Garcilaso', 'Jr. Ucayal...",['-77.0290530999999987216142471879720687866210...
7,Zimoun,ExposiciÃ³n,Espacio FundaciÃ³n TelefÃ³nica,10:00 am,Del 22.11.2018 al 10.03.2019,GRATIS,"['Espacio FundaciÃ³n TelefÃ³nica', 'Av. Arequi...",['-77.0353209000000020978404791094362735748291...
8,Taivaanranta - Firmamento,ExposiciÃ³n,Centro Cultural PUCP,10:00 am,Del 18.10.2018 al 31.01.2019,GRATIS,"['Centro Cultural PUCP', 'Av. Camino Real 1075...",['-77.0386916000000070425812737084925174713134...
9,AutotÃ©lico,ExposiciÃ³n,Casa Fugaz,11:00 am,Del 24.11.2018 al 24.01.2019,GRATIS,"['Casa Fugaz', 'Jr. ConstituciÃ³n 250, Monumen...",['-77.1473139999999943938746582716703414916992...


In [129]:
eventos_prueba=['https://www.enlima.pe/agenda-cultural/cine/el-asesinato-de-la-familia-borden', 
                'https://www.enlima.pe/agenda-cultural/exposicion/papeles-sobre-el-suelo', 'https://www.enlima.pe/agenda-cultural/exposicion/isla-azul-laminas-de-iquitos', 
                'https://www.enlima.pe/agenda-cultural/exposicion/zimoun']

In [132]:
dic_prueba = scrap_me_los_eventos(eventos_prueba)
dic_prueba

{'evento1': {'nombre_evento': 'El asesinato de la familia Borden',
  'tipo_evento': 'Cine',
  'lugar_evento': 'Varias sedes - Lima',
  'intervalo_fecha': 'Del 20.12.2018 al 03.01.2019',
  'precio': 'S/. 20.00',
  'descripcion_basica': 'Chloë Sevigny y Kristen Stewart protagonizan El asesinato de la familia Borden, una nueva adaptación sobre la conocida y presunta “asesina” del hacha, Lizzie Borden. El largometraje está a cargo de un debutante Craig William Macneil. La película presenta nuevos elementos que la diferencian de las anteriores, incluyendo la vida sentimental de Lizzie.\nLa cinta relata la vida de Lizzie Borden (Segivny), una mujer sometida al férreo control de su padre y con una vida bastante reprimida. Cuando Bridget Sullivan (Stewart) entra a vivir con la familia en busca desesperada por trabajo, Lizzie se siente atraída hacia la joven, lo que despierta en ella el deseo por elaborar un plan que la libere del su padre y así poder vivir libremente\nWilliam Macneil escogió p

#### To do: 

- Una función a la que le dé dos fechas y me bote una lista con todos los días dentro de esa fecha.  **LISTO**

- Una función a la que le pase la lista de fechas y me scrapee todos los links que hay dentro de esas fechas. **LISTO**

- Una función última de scrapper que pueda jalarme toda la información de las webs. El formato será tabular y en csv. (avanzar hasta tener un mvp e identificar los campos que tendré. También scrapear links)  **Avanzado a un 80%**

In [94]:
dict_1 = make_internal_links(lista_times[:2])

In [95]:
dict_1

{'2019-01-01': [['https://www.enlima.pe//agenda-cultural/cine/el-asesinato-de-la-familia-borden'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/papeles-sobre-el-suelo'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/liliane-tomasko'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/dora-mayer-la-perseverancia-femenina-frente-a-su-tiempo'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/caleidoscopio-de-un-arenal'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/daido-tokyo-color'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/isla-azul-laminas-de-iquitos'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/taivaanranta-firmamento'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/zimoun'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/organum-universos-corporales'],
  ['https://www.enlima.pe//agenda-cultural/exposicion/la-otra-mirada-del-jugador-dibujos-de-julio-hevia'],
  ['https://www.enlima.pe//agenda-cultural/exposicio

In [31]:
link = 'https://www.enlima.pe/calendario-cultural/dia/'+'2020-01-14'+'?page=1' 
response = requests.get(link)
soup = BeautifulSoup(response.text,"html.parser")

In [32]:
lista_links_internos = []
for i in soup.find('div',{'class':'innera'}).find_all('td',{'class':'views-field views-field-title views-align-left'}):
    link_interno = [a['href'] for a in i.find_all('a',href=True)]
    lista_links_internos.append(link_interno)

In [33]:
dict_info = {}
for i in lista_links_internos:
    link_entero = 'https://www.enlima.pe' + str(i[0])

'https://www.enlima.pe/agenda-cultural/teatro/mi-nombre-es-rachel-corrie-0'

In [71]:
two_links_internos = ['https://www.enlima.pe/agenda-cultural/teatro/mi-nombre-es-rachel-corrie-0',
                     'https://www.enlima.pe/agenda-cultural/exposicion/calendarios-japoneses']


In [76]:
dict_3 = scrap_me_internal_links(['https://www.enlima.pe/calendario-cultural/dia/2019-01-01?page=1',
 'https://www.enlima.pe/calendario-cultural/dia/2019-01-02?page=1'])

https://www.enlima.pe/agenda-cultural/cine/el-asesinato-de-la-familia-borden
https://www.enlima.pe/agenda-cultural/exposicion/papeles-sobre-el-suelo
https://www.enlima.pe/agenda-cultural/exposicion/daido-tokyo-color
https://www.enlima.pe/agenda-cultural/exposicion/isla-azul-laminas-de-iquitos
https://www.enlima.pe/agenda-cultural/exposicion/taivaanranta-firmamento
https://www.enlima.pe/agenda-cultural/exposicion/zimoun
https://www.enlima.pe/agenda-cultural/exposicion/liliane-tomasko
https://www.enlima.pe/agenda-cultural/exposicion/dora-mayer-la-perseverancia-femenina-frente-a-su-tiempo
https://www.enlima.pe/agenda-cultural/exposicion/caleidoscopio-de-un-arenal
https://www.enlima.pe/agenda-cultural/exposicion/organum-universos-corporales
https://www.enlima.pe/agenda-cultural/exposicion/la-otra-mirada-del-jugador-dibujos-de-julio-hevia
https://www.enlima.pe/agenda-cultural/exposicion/autotelico
https://www.enlima.pe/agenda-cultural/cine/el-asesinato-de-la-familia-borden
https://www.enlim

In [77]:
dict_3

{'2019-01-01': {'evento1': {'nombre_evento': 'Autotélico',
   'tipo_evento': 'Exposición',
   'lugar_evento': 'Casa Fugaz',
   'intervalo_fecha': '11:00 am',
   'precio': 'Del 24.11.2018 al 24.01.2019',
   'descripcion_basica': 'GRATIS',
   'data_extra': ['Casa Fugaz',
    'Jr. Constitución 250, Monumental Callao',
    'Callao',
    '+ info',
    ''],
   'coordenadas': ['-77.14731399999999439387465827167034149169921875',
    '-12.059886999999999801502781338058412075042724609375']}},
 '2019-01-02': {'evento1': {'nombre_evento': 'Reina por un día',
   'tipo_evento': 'Teatro Marsano',
   'lugar_evento': '8:00 pm',
   'intervalo_fecha': 'Del 02.01.2019 al 30.03.2019',
   'precio': 'S/. 38.00 a S/. 80.00',
   'descripcion_basica': 'Valentina es\xa0una madre que abandona a su hijo al nacer, quien crece con un sentimiento permanente de frustración y resentimiento hacia ella.\xa0Veinticinco años después, en un Día de la Madre, Valentina regresa a la casa donde vive su hijo Jorge y su esposa, p

In [73]:
scrap_me_all('https://www.enlima.pe/agenda-cultural/exposicion/calendarios-japoneses')

In [74]:
dict_scrap_final

{'evento1': {'nombre_evento': 'Calendarios japoneses',
  'tipo_evento': 'Centro Cultural Peruano Japonés',
  'lugar_evento': '9:00 am',
  'intervalo_fecha': 'Del 06.01.2020 al 31.01.2020',
  'precio': 'GRATIS',
  'descripcion_basica': 'La Embajada del Japón en el Perú y la Asociación Peruano Japonesa presentan esta tradicional muestra de almanaques japoneses. La muestra reúne ejemplares con bellos diseños basados en las tradiciones japonesas, que los apartan de su función utilitaria para convertirlos en objetos artísticos de gran atractivo.\nEsta exposición se puede visitar del 6 al 31 de enero 2020, todos los días de 9 am a 9 pm.\nINGRESO LIBRE\n',
  'data_extra': ['Centro Cultural Peruano Japonés',
   'Calle Gregorio Escobedo 803, Residencial San Felipe',
   'Jesús María',
   'lun-dom 9am-9pm\n',
   '518-7450',
   '+ info',
   ''],
  'coordenadas': ['-77.055456899999995812322595156729221343994140625',
   '-12.08798110000000036734491004608571529388427734375']}}

In [44]:
response = requests.get('https://www.enlima.pe/agenda-cultural/exposicion/calendarios-japoneses')
soup = BeautifulSoup(response.text,"html.parser")

In [45]:
#Titulo
soup.find('h1',{'class':'page__title title'}).text

'Calendarios japoneses'

In [46]:
# tipo, lugar, intervalo_fecha, precio, descripcion_basica
basic_info = [div.text for div in soup.find_all('div',{'class':'field-item even'})[2:7]]
tipo, lugar, intervalo_fecha, precio, descripcion_basica = map(str,basic_info)

In [49]:
print(tipo,'--', lugar,'--', intervalo_fecha,'--', precio,'--', descripcion_basica)

Centro Cultural Peruano Japonés -- 9:00 am -- Del 06.01.2020 al 31.01.2020 -- GRATIS -- La Embajada del Japón en el Perú y la Asociación Peruano Japonesa presentan esta tradicional muestra de almanaques japoneses. La muestra reúne ejemplares con bellos diseños basados en las tradiciones japonesas, que los apartan de su función utilitaria para convertirlos en objetos artísticos de gran atractivo.
Esta exposición se puede visitar del 6 al 31 de enero 2020, todos los días de 9 am a 9 pm.
INGRESO LIBRE



In [50]:
# Direccion, Distrito, horarios y dias, numero
#OJO, no todos tienen estos campos. Por eso lo pondré para parsear luego. Que me encuentre todo lo que pueda. 
to_parse = [elem.text for elem in soup.find('div',{'class':'ds-1col node node-lugar view-mode-reference clearfix'}).find_all('div',{'class':'field-item even'})]
to_parse

['Centro Cultural Peruano Japonés',
 'Calle Gregorio Escobedo 803, Residencial San Felipe',
 'Jesús María',
 'lun-dom 9am-9pm\n',
 '518-7450',
 '+ info',
 '']

In [69]:
#Longitud y latitud
# Algunos tienen otros no. Entonces se recomienda hacer un try except... probar con los que no tienen y sí. 
import re

pattern = re.compile(r'-\d*.\d*,-\d*.\d*',re.MULTILINE|re.DOTALL) #Tengo el patron de las coordenadas

script = soup.find_all('script',text=pattern) #saco el script que contiene el patron

coordenadas_list = re.findall(pattern,str(script))[0].split(',') #Lo obtengo y lo pongo dentro de una lista. 

In [70]:
coordenadas_list

['-77.055456899999995812322595156729221343994140625',
 '-12.08798110000000036734491004608571529388427734375']